## Recommendation System - Part I
### Product popularity based recommendation system targeted at new customers
* Popularity based are a great strategy to target the new customers with the most popular products sold on a business's website and is very useful to cold start a recommendation engine.
[[](http://)](http://)
* **Dataset : **[Amazon product review dataset](https://www.kaggle.com/skillsmuggler/amazon-ratings)

#### Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %matplotlib inline
plt.style.use("ggplot")

import sklearn
from sklearn.decomposition import TruncatedSVD

#### Loading the dataset

In [2]:
amazon_ratings = pd.read_csv('../input/d/nahushbhamre/amazon-ratings/ratings_Electronics.csv')
amazon_ratings.columns = ["UserId", "ProductId", "Rating", "Timestamp"]
amazon_ratings = amazon_ratings.dropna()
amazon_ratings.head()

In [3]:
amazon_ratings.shape

In [4]:
popular_products = pd.DataFrame(amazon_ratings.groupby('ProductId')['Rating'].count())
most_popular = popular_products.sort_values('Rating', ascending=False)
most_popular.head(10)

In [5]:
# Most popular products (arranged in descending order) sold by the business.

most_popular.head(25).plot(kind = "bar")

## Recommendation System - Part II
### Model-based collaborative filtering system

* Recommend items to users based on purchase history and similarity of ratings provided by other users who bought items to that of a particular customer.
* A model based collaborative filtering technique is closen here as it helps in making predictinfg products for a particular user by identifying patterns based on preferences from multiple user data.

#### Utility Matrix based on products sold and user reviews
**Utility Matrix : **An utlity matrix is consists of all possible user-item preferences (ratings) details represented as a matrix. The utility matrix is sparce as none of the users would buy all teh items in the list, hence, most of the values are unknown.

In [6]:
# Subset of Amazon Ratings

# amazon_ratings1 = amazon_ratings
amazon_ratings1 = amazon_ratings.head(75000)

In [7]:
ratings_utility_matrix = amazon_ratings1.pivot_table(values='Rating', index='UserId', columns='ProductId', fill_value=0)
ratings_utility_matrix.head()

The utility matrix obtaned above is sparce, the unknown values have been filled with 0.

In [8]:
ratings_utility_matrix.shape

Transposing the matrix

In [9]:
X = ratings_utility_matrix.T
X.head()

Unique products in subset of data

### Decomposing the Matrix

In [10]:
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

### Correlation Matrix

In [11]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

correlation_matrix

### Isolating Product ID # 1400698987 from the Correlation Matrix

Assuming the customer buys Product ID # 1400698987 (randomly chosen)

In [12]:
X.index[150]

Index # of product ID purchased by customer

In [13]:
i = "1400698987"

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

Correlation for all items with the item purchased by this customer based on items rated by other customers people who bought the same product

In [14]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

### Recommending top 10 highly correlated products in sequence

In [15]:
Recommend = list(X.index[correlation_product_ID > 0.90])

# Removes the item already bought by the customer
Recommend.remove(i) 

Recommend[0:9]

**Product Id # **
Here are the top 10 products to be displayed by the recommendation system to the above customer based on the purchase history of other customers in the website. 

## Recommendation System - Part III
* For a business without any user-item purchase history, a search engine based recommendation system can be designed for users. The product recommendations can be based on textual clustering analysis given in product description.
* **Dataset : **[Home Depot's dataset with product dataset.](https://www.kaggle.com/c/home-depot-product-search-relevance/data)

In [16]:
# Importing libraries

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

### Item to item based recommendation system based on product description

Applicable when business is setting up its E-commerce website for the first time

In [17]:
product_descriptions = pd.read_csv('../input/home-depot-product-search-relevance/product_descriptions.csv.zip')
# product_descriptions = pd.read_csv('../input/flipkart/flipkart_com-ecommerce_sample.csv')
product_descriptions.shape

#### Checking for missing values

In [18]:
# Missing values

product_descriptions = product_descriptions.dropna()
product_descriptions.shape
product_descriptions.head()

In [19]:
product_descriptions.shape

In [42]:
product_descriptions1 = product_descriptions.head(25000)
# product_descriptions1 = product_descriptions
# product_descriptions1.iloc[:,1]

product_descriptions1["product_description"].head(10)

#### Feature extraction from product descriptions

Converting the text in product description into numerical data for analysis

In [43]:
vectorizer = TfidfVectorizer(stop_words='english')
X1 = vectorizer.fit_transform(product_descriptions1["product_description"])
X1

#### Visualizing product clusters in subset of data

In [44]:
# Fitting K-Means to the dataset

X=X1

kmeans = KMeans(n_clusters = 10, init = 'k-means++')
y_kmeans = kmeans.fit_predict(X)
plt.plot(y_kmeans, ".")
plt.show()

In [45]:
def print_cluster(i):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

## Output
* Recommendation of product based on the current product selected by user.
* To recommend related product based on, Frequently bought together. 

#### Top words in each cluster based on product description

In [46]:
# Optimal clusters is...

true_k = 10

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X1)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print_cluster(i)

#### Predicting clusters based on key search words

In [47]:
def show_recommendations(product):
    #print("Cluster ID:")
    Y = vectorizer.transform([product])
    prediction = model.predict(Y)
    #print(prediction)
    print_cluster(prediction[0])

* **Keyword : ** cutting tool

In [48]:
show_recommendations("home energy")

* **Keyword : **spray paint

In [49]:
show_recommendations("water pump")

* **Keyword : **steel drill

In [50]:
show_recommendations("steel drill")

In case a word appears in multiple clusters, the algorithm chooses the cluster with the highest frequency of occurance of the word.

* **Keyword : **water

In [51]:
show_recommendations("water")

Once a cluster is identified based on the user's search words, the recommendation system can display items from the corresponding product clusters based on the product descriptions.